In [2]:
import torch
import os
from tqdm import tqdm
import torch.optim as optim
import numpy as np
from sklearn.metrics import precision_recall_curve
# user written
from bcosTransformer import BCosTransformer
from torch.utils.data import Dataset,ConcatDataset,DataLoader,random_split
import pandas as pd
from dataloader import CODE2Dataset, BatchDataloader, CODE2Dataloader
import utils
from metrics import EcgMetrics
import h5py

In [3]:
path_to_csv = "/srv/derickmath/aws/h5files/classification_data.csv"
path_to_h5 = "/srv/derickmath/aws/h5files/metadata_all_months.h5"


h5file = h5py.File(path_to_h5, 'r')
train_dataset = CODE2Dataloader(h5file, path_to_csv)
train_loader = DataLoader(dataset=train_dataset,batch_size=32,num_workers=1)

In [9]:
idx = 5983
new_idx = train_dataset.train[idx]
X=train_dataset.traces[new_idx]
traces = np.empty(shape = (train_dataset.traces.shape[0], 1024))
try:
    segment = nk.ecg_peaks(X[0], sampling_rate=400)
    Rpeak = segment[1]['ECG_R_Peaks'][1]
    if Rpeak < 4096 - 1024:
        traces = X[:,Rpeak:Rpeak+1024]
    else:
        traces = X[:,0:1024]

except:
    traces = X[:,0:1024]
X=torch.tensor(traces)

In [10]:
X.shape

torch.Size([8, 1024])

In [3]:
path_to_h5 = "/home/derickmath/UFMG/CODE/data/exams_part13.hdf5"
path_to_csv = "/home/derickmath/UFMG/CODE/data/exams_formated.csv"

# dset = CODE2Dataset(
#         path_to_h5,
#         path_to_csv, traces_dset="tracings", exam_id_dset="exam_id"
#     )
# n_valid = sum(dset.val)
# valid_loader = BatchDataloader(dset, batch_size, mask=dset.val)
# train_loader = BatchDataloader(dset, batch_size, mask=dset.train)

In [4]:
import h5py

f = h5py.File(path_to_h5)
x = f['tracings'][0:10]
model = BCosTransformer(dim = 4096, depth=1, heads=2, in_features=4096, nclass=6, dim_head=2)
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
x = torch.tensor(x[:,:,0]).to(device)

In [6]:
model(x)

tensor([[-1.2340e+00,  2.0559e-01,  5.8346e-01,  2.1753e+00, -1.7350e+00,
         -2.1304e+00],
        [-1.2632e+00,  2.2897e-01,  5.5067e-01,  2.3123e+00, -1.5513e+00,
         -1.8453e+00],
        [-1.2398e+00,  1.9379e-01,  5.7362e-01,  2.1993e+00, -1.7574e+00,
         -2.1470e+00],
        [-9.8802e-03, -1.3189e-02, -3.3579e-02, -4.9052e-02,  3.3714e-08,
         -3.4260e-02],
        [-1.3000e+00,  2.1143e-01,  6.0529e-01,  2.2899e+00, -1.7848e+00,
         -2.1878e+00],
        [-1.2334e+00,  2.1018e-01,  6.0926e-01,  2.1962e+00, -1.7760e+00,
         -2.1937e+00],
        [-1.2133e+00,  1.9648e-01,  5.6900e-01,  2.1301e+00, -1.7035e+00,
         -2.0991e+00],
        [-1.1588e+00,  1.8948e-01,  5.4935e-01,  2.0428e+00, -1.6150e+00,
         -2.0057e+00],
        [-1.2643e+00,  1.9848e-01,  5.9025e-01,  2.2188e+00, -1.7663e+00,
         -2.1623e+00],
        [-1.2246e+00,  1.8831e-01,  5.7496e-01,  2.1447e+00, -1.7027e+00,
         -2.0906e+00]], device='cuda:0', grad_fn=<Div